<a href="https://colab.research.google.com/github/pinilDissanayaka/Election-Insight-App/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [15]:
import os
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_groq.chat_models import ChatGroq
from langchain_experimental.text_splitter import SemanticChunker
from langchain.document_loaders import PyPDFLoader
from langchain_pinecone import PineconeVectorStore
from typing import List, TypedDict

In [6]:
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECORN_API_KEY')
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY']=userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']='true'

In [ ]:
constitution_path="/content/constitution.pdf"

loader=PyPDFLoader(file_path=constitution_path)

constitution_data=loader.load()

print(f"{len(constitution_data)} documents are loaded.")
print("="*1000)
print("A sample document is:")
print(constitution_data[6].page_content)

In [8]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings=HuggingFaceEmbeddings(model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
constitution_data_chunks=SemanticChunker(embeddings=embeddings).split_documents(constitution_data)

print(f"{len(constitution_data_chunks)} chunks are created.")

601 chunks are created.


In [ ]:
def createIndex(indexName:str, dimension:int):
  try:
    pinecone=Pinecone()
    if indexName not in pinecone.list_indexes().names():
      pinecone.create_index(
          name=indexName,
          dimension=dimension,
          metric="cosine",
          spec=ServerlessSpec(cloud='aws',
                            region='us-east-1')
      )
      print(f"Created {indexName}")
    else:
      print(f"{indexName} already exists")
    print(pinecone.describe_index(indexName))
    return pinecone.Index(indexName)
  except Exception as e:
    print(e.args)

In [ ]:
dimention=768

In [ ]:
index=createIndex(indexName="constitution", dimension=dimention)

constitution already exists
{'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'constitution-4myrn7y.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'constitution',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [ ]:
pinecone_vectore_store=PineconeVectorStore.from_documents(documents=constitution_data_chunks, embedding=embeddings, index_name="constitution")

In [10]:
pinecone_vectore_store=PineconeVectorStore(embedding=embeddings, index_name="constitution")
retriever=pinecone_vectore_store.as_retriever()

In [14]:
class GraphState(TypedDict):
